In [1]:
import sys
import os

import pynq

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from Biquad.Biquad_Daq import Biquad_Daq
from Biquad.SimBiquad import SimBiquad

from Utils.PlotTools import PlottingTools as pt

from Waveforms.Waveform import Waveform
from Waveforms.Gated import Gated
from Waveforms.Filterred import Filterred

In [2]:
daq = Biquad_Daq(None, None, 4, 2**10)

Turning on SYNC
Turning off SYNC


In [3]:
def update_plot(daq, A, B, P, theta):
    fig, (axD,axS) = plt.subplots(2, 1, figsize=(25, 20))
    
    daq.reset()
    
    daq.setA(A)
    daq.setB(B)
    daq.setP(P)
    daq.setTheta(np.pi*theta)
    
    
    daq.set_single_zero_fir()
    daq.set_f_fir(0)
    daq.set_g_fir(0)
    # daq.set_F_fir()
    # daq.set_G_fir()
    # daq.set_pole_coef()
    # daq.set_incremental()

    daq.JupyterAcquire()
    
    daq.waveforms[0].setRMS()
    amp = daq.waveforms[0].RMS
    
    biquad = SimBiquad(data=daq.adcBuffer[0] >> 4, A=A, B=B, P=P, theta=np.pi*theta, amp = amp)  
    biquad.set_daq_coeffs(daq.get_coeffs())
    
    biquad.single_zero_fir()     #.get_u()
    biquad.first_constants()    #.get_decimated1()
    biquad.second_constants()    #.get_decimated2()
    biquad.IIR_calculation()   #get_decimated
    # biquad.implementation() #get_biquad()
        
    sim_output = biquad.get_decimated1()
    
    ##This needs to be rewritten because I've 'automated' the setting of the waveform classes
    # diff_output = Filterred(np.array(daq.waveforms[2].waveform)-biquad.get_decimated1()[280:880], 0, None)
    
    daq.waveforms[2].plotWaveform(axD, title="Biquaded ADC224_T0_CH0")
    sim_output.plotWaveform(axS, title="Simulated Biquad")
    
    
    
    # print(sim_output.calc_Lambda() * sim_output.R_f() * amp)
    # print(sim_output.calc_Lambda() * sim_output.R_g() * amp)
    print(sim_output.calc_Lambda() * sim_output.Average_R_fg() * amp)
    
    daq.waveforms[2].setRMS()
    print(daq.waveforms[2].RMS)
    
    # daq.printParams()
    # print('')
    # daq.printCoeffs()
        
    plt.show()

In [4]:
A_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.1, value=1, description='A:', continuous_update=False)
B_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.1, value=1, description='B:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=0.99, step=0.01, value=0.62, description='P:', continuous_update=False)
Theta_slider = widgets.FloatSlider(min=0, max=2, step=0.01, value=0.95, description='Theta:', continuous_update=False)
    
plot_output = widgets.Output()

def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        update_plot(daq, A_slider.value, B_slider.value, P_slider.value, Theta_slider.value)
    
A_slider.observe(update, names='value')
B_slider.observe(update, names='value')
P_slider.observe(update, names='value')
Theta_slider.observe(update, names='value')

slider_box = widgets.HBox([A_slider, B_slider, P_slider, Theta_slider])
ui = widgets.VBox([slider_box, plot_output])

display(ui)

# Initial plot
update(None)

In [5]:
# A=1
# B=1
# P=0.62
# Theta=0.95
# The predicted amplituded isn't resolving properly